1. Change notebook name
2. Change df name (80 occurances)
3. Change json file name
4. Set collected on date


In [3]:
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime
import numpy as np
import timeit
import json

In [4]:
start_time = timeit.default_timer()


df_name = pd.read_json('json_datasets/Kickstarterxxxx', lines=True)


elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

ValueError: Expected object or value

In [ ]:
# Save the creation datetime for calculating age of project later
df_name_collected_on = datetime(xxxx, xx, xx)

In [ ]:
# clean up the format and make a deep copy
start_time = timeit.default_timer()


df_name = df_name['data'].apply(pd.Series)
df_name_copy = df_name.copy(deep=True)
print(f'Total_rows: {len(df_name)}')


elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

df_name.head(3)

In [ ]:
# Check if data has same features
df_name_cols = df_name.columns
columns_original  = ['id', 'photo', 'name', 'blurb', 'goal', 'pledged', 'state', 'slug', 'disable_communication', 'country', 'country_displayable_name', 'currency', 'currency_symbol', 'currency_trailing_code', 'deadline', 'state_changed_at', 'created_at', 'launched_at', 'staff_pick', 'is_starrable', 'backers_count', 'static_usd_rate', 'usd_pledged', 'converted_pledged_amount', 'fx_rate', 'current_currency', 'usd_type', 'creator', 'location', 'category', 'profile', 'spotlight', 'urls', 'source_url', 'friends', 'is_starred', 'is_backing', 'permissions']
diff = list(set(df_name_cols) - set(columns_original))
diff2 = list(set(columns_original) - set(df_name_cols))
print(diff)
print(diff2)

In [ ]:
# inspect canceled and suspended states
df_name.state.value_counts()

In [ ]:
# drop canceled and suspended states
df_name.drop((df_name[((df_name.state == 'suspended') | 
                                         (df_name.state == 'canceled'))].index), inplace=True)

# check for duplicates
print(f"Rows with duplicate project_id: {df_name.duplicated(['id']).sum()}")

# drop duplicates
df_name.drop_duplicates(['id'], inplace=True)
print(f'Total_rows after dropping duplicates and suspended/canceled states: {len(df_name)}')


In [ ]:
# get links to project and creator pages
start_time = timeit.default_timer()


df_name['project_link'] = 'no_val'
project_link_loc = df_name.columns.get_loc('project_link')
for n in range(len(df_name)):
    project_link = df_name.iloc[n].urls['web']['project']
    df_name.iat[n, project_link_loc] = project_link
    
df_name['creator_link'] = 'no_val'
creator_link_loc = df_name.columns.get_loc('creator_link')
for n in range(len(df_name)):
    creator_link = df_name.iloc[n].creator['urls']['web']['user']
    df_name.iat[n, creator_link_loc] = creator_link  
    
    
elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

In [ ]:
# get main_category id
start_time = timeit.default_timer()


categories_map = {1:'Art',3:'Comics', 17:'Theater', 15:'Photography', 
                7:'Design', 6: 'Dance', 11:'Film & Video', 12:'Games', 16:'Technology', 
                13:'Journalism', 18:'Publishing', 9:'Fashion', 10:'Food', 14:'Music', 26:'Crafts'}
categories_df = pd.DataFrame(categories_map, index=[0])


df_name['main_category'] = 'no_value'
cat_loc = df_name.columns.get_loc('main_category')


for n in (range(len(df_name))):
    if 'parent_id' not in df_name.iloc[n]['category'].keys():
        cat_id = df_name.iloc[n]['category']['id']
        df_name.iat[n, cat_loc] = cat_id
    elif pd.isnull(df_name.iloc[n]['category']['parent_id']):
        cat_id = df_name.iloc[n]['category']['id']
        df_name.iat[n, cat_loc] = cat_id
    else:
        par_id = df_name.iloc[n]['category']['parent_id']
        df_name.iat[n, cat_loc] = par_id
        
df_name['main_category'] = df_name['main_category'].apply(lambda x: categories_df[x])


elapsed = timeit.default_timer() - start_time
print(f'time: {elapsed}')

df_name.head(1)

In [ ]:
# keep these features
df_main_feats = ['id', 'name', 'blurb', 'goal', 'pledged', 'state', 'country',
'currency', 'deadline', 'launched_at', 'staff_pick', 'main_category',
'backers_count', 'static_usd_rate', 'usd_pledged' ]
# Drop these 
df_drop_feats = list(set(df_name_cols) - set(df_main_feats))
print(f'Columns removed: {df_drop_feats}')

# drop features
df_name.drop(df_drop_feats, axis=1, inplace=True)

In [ ]:
# convert unix dates to datetime objects
df_name.deadline = df_name.deadline.apply(lambda x: (datetime.fromtimestamp(x)))
df_name.launched_at = df_name.launched_at.apply(lambda x: (datetime.fromtimestamp(x)))

In [ ]:
# rename columns
df_name.rename(columns={'id':'project_id'}, inplace=True)
df_name.rename(columns={'name':'project_name'}, inplace=True)
df_name.rename(columns={'blurb':'project_description'}, inplace=True)
df_name.rename(columns={'state':'outcome_state'}, inplace=True)


In [ ]:
# add project duration feature
df_name['duration'] = df_name.deadline - df_name.launched_at
df_name['days_from_launch'] =(df_name_collected_on - df_name.launched_at).apply(lambda x: x.days)

# add feature 'in_mid_duration_range' indicating whether project is about halfway through
df_name['duration_mid_point'] = df_name.duration.apply(lambda x: x.days) / 2
df_name['lower_bound_mid_point_range'] = df_name.duration_mid_point.apply(lambda x: x - 5)
df_name['upper_bound_mid_point_range'] = df_name.duration_mid_point.apply(lambda x: x + 5)
df_name['in_mid_duration_range'] = df_name.days_from_launch.between(df_name.lower_bound_mid_point_range, df_name.upper_bound_mid_point_range)

df_name.drop(['duration_mid_point', 'lower_bound_mid_point_range', 'upper_bound_mid_point_range'], axis=1, inplace=True)


In [ ]:
# calculate usd_goal
df_name['usd_goal'] = df_name.goal * df_name.static_usd_rate


In [ ]:
# Replace goal and rearrange
col_list = df_name.columns.to_list()
print(col_list)
col_list_rearrange = ['project_id', 'project_name', 'project_description', 'outcome_state', 'in_mid_duration_range',
                      'usd_goal', 'usd_pledged', 'currency', 'launched_at', 'deadline', 'staff_pick', 'country', 'main_category', 
                      'duration' , 'days_from_launch', 'backers_count', 'project_link', 'creator_link']
df_name = df_name[col_list_rearrange]
df_name.head(1)
                      

In [ ]:
# convert datetime columns before exporting
df_name.duration = df_name.duration.apply(lambda x: x.days) 
df_name.launched_at = df_name.launched_at.apply(lambda x: x.strftime('%m-%d-%Y'))
df_name.deadline = df_name.deadline.apply(lambda x: x.strftime('%m-%d-%Y'))

In [ ]:
# export cleaned dataframe to csv
df_name.to_csv('clean_datasets/main_df_name.csv', index=False)